# Gemini Tool-Calling Chatbot

This notebook implements a chatbot using Google Gemini 2.0 Flash with function calling capabilities (Weather tool). 

In [ ]:
import os
import requests
import google.generativeai as genai
from google.generativeai.types import FunctionDeclaration, Tool
import ipywidgets as widgets
from IPython.display import display, Markdown
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
WEATHER_API_KEY = os.getenv('WEATHER_API_KEY')

if not GEMINI_API_KEY or not WEATHER_API_KEY:
    print("Error: API Keys not found in .env file!")
else:
    genai.configure(api_key=GEMINI_API_KEY)
    print("API Keys loaded successfully.")

In [ ]:
# Define Tools

def get_weather(city: str):
    """Get the current weather for a specific city name."""
    print(f"[Tool] Fetching weather for {city}...")
    api_key = WEATHER_API_KEY
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            return {
                "temperature": f"{data['main']['temp']}°C",
                "condition": data['weather'][0]['description'],
                "humidity": f"{data['main']['humidity']}%"
            }
        else:
            return {"error": f"Weather API Error: {response.text}"}
    except Exception as e:
        return {"error": str(e)}

tools_list = [get_weather]
model = genai.GenerativeModel(
    model_name='gemini-2.0-flash',
    tools=tools_list
)
chat = model.start_chat(enable_automatic_function_calling=True)

In [ ]:
# Create UI

output_area = widgets.Output(layout={'border': '1px solid #ccc', 'height': '400px', 'overflow_y': 'scroll', 'padding': '10px'})
input_box = widgets.Text(placeholder='Ask about weather...', layout={'width': '80%'})
send_btn = widgets.Button(description='Send', button_style='primary')

def on_send(b):
    user_msg = input_box.value
    if not user_msg: return
    
    input_box.value = ''
    with output_area:
        print(f"You: {user_msg}")
        display(Markdown("*Thinking...*"))
    
    try:
        response = chat.send_message(user_msg)
        with output_area:
            # Clear the "Thinking..." line is tricky in simple Output widgets, so we just print the response.
            display(Markdown(f"**Gemini**: {response.text}"))
            print("-" * 40)
    except Exception as e:
        with output_area:
            print(f"Error: {e}")

send_btn.on_click(on_send)
input_box.on_submit(on_send)

ui = widgets.VBox([output_area, widgets.HBox([input_box, send_btn])])
display(ui)